In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista
import ufl
import time
from mpi4py import MPI
from petsc4py import PETSc
from dolfinx import fem, mesh, plot, nls, log, io
from dolfinx import cpp as _cpp
import meshio
import os
import seaborn as sns
import json
%load_ext autoreload
%autoreload 2

from utils import *
from model import *

with io.XDMFFile(MPI.COMM_WORLD, '../../raw-data/small_mesh/rve_small.xdmf', 'r') as xdmf:
    domain = xdmf.read_mesh(name='Grid')

results_folder = r"../../results-data/"
exp_folder = "small_mesh"

model = PerfusionGasExchangeModel(mesh_path=None,
                                  results_path=results_folder,
                                  exp_path=exp_folder,
                                  params=None)

model.Setup(domain)
model.parameter_setup()

max_dims = [24.76658 24.74611 24.77482]
min_dims = [0.28539 0.2329  0.2496 ]


In [3]:
### Single loop test (works!)



model.p_params["uin"] = 25/0.75
p, u = model.Perfusion(domain, plot=False, save=True)

p_val = 0
guess = model.GasExchange(domain, guess=None, save=True, 
                          plot=False, p_val = p_val, 
                          postprocess=False, plot_lines=False)
N = 20
for i in range(N):
    p_val += 1/N
    if i != N-1:
        guess = model.GasExchange(domain, guess=guess, save=False, 
                                  plot=False, p_val = p_val, 
                                  postprocess=False, plot_lines=False)
    else:
        guess = model.GasExchange(domain, guess=guess, save=True, 
                                  plot=False, p_val = p_val, 
                                  postprocess=True, plot_lines=True)

2023-12-19 16:12:19.169 ( 215.821s) [main            ]                MPI.cpp:166   INFO| Computing communication graph edges (using NBX algorithm). Number of input edges: 0
2023-12-19 16:12:19.170 ( 215.823s) [main            ]                MPI.cpp:237   INFO| Finished graph edge discovery using NBX algorithm. Number of discovered edges 0
2023-12-19 16:12:19.171 ( 215.824s) [main            ]                MPI.cpp:166   INFO| Computing communication graph edges (using NBX algorithm). Number of input edges: 0
2023-12-19 16:12:19.171 ( 215.824s) [main            ]                MPI.cpp:237   INFO| Finished graph edge discovery using NBX algorithm. Number of discovered edges 0
2023-12-19 16:12:19.171 ( 215.824s) [main            ]                MPI.cpp:166   INFO| Computing communication graph edges (using NBX algorithm). Number of input edges: 0
2023-12-19 16:12:19.171 ( 215.824s) [main            ]                MPI.cpp:237   INFO| Finished graph edge discovery using NBX algorith

Problem instanced.
Pressure field found.
Mesh path: ../../results-data/small_mesh/p_new
Mesh path: ../../results-data/small_mesh/u_new
Problem solved in 2 iterations.
Mesh path: ../../results-data/small_mesh/gas_exchange/
Mesh path: ../../results-data/small_mesh/gas_exchange/


2023-12-19 16:12:20.306 ( 216.958s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-12-19 16:12:20.345 ( 216.998s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-12-19 16:12:20.358 ( 217.011s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 0 (tol = 1e-08) r (rel) = -nan(tol = 1e-08)
2023-12-19 16:12:20.358 ( 217.011s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 2 iterations and 0 linear solver iterations.


Starting nonlinear problem with previous guess.
p_val = 0.1


2023-12-19 16:12:23.598 ( 220.250s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-12-19 16:12:23.722 ( 220.374s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-12-19 16:12:23.739 ( 220.391s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = inf (tol = 1e-08) r (rel) = -nan(tol = 1e-08)
2023-12-19 16:12:23.827 ( 220.479s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-12-19 16:12:23.845 ( 220.497s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = inf (tol = 1e-08) r (rel) = -nan(tol = 1e-08)
2023-12-19 16:12:23.927 ( 220.580s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-12-19 16:12:23.944 ( 220.596s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4: r (abs) = inf 

RuntimeError: Newton solver did not converge because maximum number of iterations reached